In [ ]:
import csv  # imports CSV module for reading and writing CSV files
import os  # imports OS module to check for file existence and interact with operating system
import datetime  # imports datetime module to work with dates

CSV_FILE = "expenses.csv"  # defines filename for storing expense data

def initialize_file():
    """
    Initialize the CSV file.
    Checks if the CSV file exists, and if it doesn't, creates it with a header row.
    """
    if not os.path.exists(CSV_FILE):  # checks if CSV file does not exist
        with open(CSV_FILE, 'w', newline='') as csvfile:  # opens file in write mode where newline prevents extra blank lines
            writer = csv.writer(csvfile)  # creates CSV writer object
            writer.writerow(["Date", "Category", "Description", "Amount"])  # writes header row to CSV file

def add_expense():
    """
    Add a new expense record.
    Prompts the user for details (date, category, description, and amount) and appends the record to the CSV file.
    """
    date = input("Enter date (YYYY-MM-DD) [press Enter for today's date]: ").strip()  # asks user for date
    if date == "":  # if no date provided...
        date = datetime.date.today().isoformat()  # defaults to today's date in ISO format
    category = input("Enter expense category: ").strip()  # prompts user for expense category
    description = input("Enter description: ").strip()  # prompts user for brief description

    while True:  # loops until valid numeric amount entered
        try:
            amount = float(input("Enter amount: ").strip())  # prompts and converts amount to a float
            break  # exits the loop if conversion is successful
        except ValueError:  # if conversion fails...
            print("Invalid amount. Please enter a numeric value.")  # informs user of the error

    with open(CSV_FILE, 'a', newline='') as csvfile:  # opens CSV file in append mode
        writer = csv.writer(csvfile)  # creates CSV writer object
        writer.writerow([date, category, description, amount])  # writes new expense record
    print("Expense added successfully!")  # confirms expense was added

def view_expenses():
    """
    Display all expense records.
    Reads and prints every row from the CSV file.
    """
    print("\nExpenses:")  # prints header
    if not os.path.exists(CSV_FILE):  # checks if CSV file exists
        print("No expenses recorded yet.")  # informs user if no expenses found
        return  # exits function if file does not exist

    with open(CSV_FILE, 'r') as csvfile:  # opens CSV file in read mode
        reader = csv.reader(csvfile)  # creates CSV reader object
        for row in reader:  # iterates over every row
            print(row)  # prints current row
    print()  # prints extra newline for formatting

def view_summary():
    """
    Summarize all expenses.
    Calculates the total spending and provides a breakdown by category.
    """
    total = 0  # initializes total spending variable
    category_totals = {}  # initializes dictionary to store spending by category
    if not os.path.exists(CSV_FILE):  # checks if CSV file exists
        print("No expenses recorded yet.")  # informs user if file does not exist
        return  # exits function

    with open(CSV_FILE, 'r') as csvfile:  # opens CSV file in read mode
        reader = csv.reader(csvfile)  # create CSV reader
        header = next(reader, None)  # skip header row
        for row in reader:  # iterates through each expense record
            if len(row) < 4:  # skips incomplete rows
                continue
            _, category, _, amount = row  # extracts category and amount from record
            try:
                amount = float(amount)  # converts amount to a float
            except ValueError:
                continue  # skips record if conversion fails
            total += amount  # adds amount to total spending
            category_totals[category] = category_totals.get(category, 0) + amount  # updates category total

    print("\nExpense Summary:")  # prints header for summary
    print(f"Total Expenses: ${total:.2f}")  # displays total spending
    print("Expenses by Category:")  # prints subheader
    for cat, amt in category_totals.items():  # loops through each category
        print(f"  {cat}: ${amt:.2f}")  # displays category and its total spending
    print()  # prints newline

# --- Divide-and-Conquer Algorithm for Maximum Subarray Analysis ---

def find_max_crossing_subarray(arr, low, mid, high):
    """
    Find the maximum subarray crossing the midpoint.
    Searches for the subarray that spans the middle and has the maximum sum.

    Parameters:
      arr  - List of daily expense totals.
      low  - Starting index of the current subarray.
      mid  - Middle index where the subarray is divided.
      high - Ending index of the current subarray.

    Returns:
      A tuple (max_left, max_right, sum) representing the best subarray's starting index, ending index, and total sum.
    """
    left_sum = float('-inf')  # initializes left-side sum to negative infinity
    total = 0  # initializes cumulative sum for left side
    max_left = mid  # sets initial left boundary as middle
    for i in range(mid, low - 1, -1):  # iterates backward from mid to low
        total += arr[i]  # adds current element to cumulative sum
        if total > left_sum:  # if current sum is greater than best left sum...
            left_sum = total  # updates best left sum
            max_left = i  # updates left boundary
    right_sum = float('-inf')  # initializes right-side sum to negative infinity
    total = 0  # resets cumulative sum for right side
    max_right = mid + 1  # sets initial right boundary
    for j in range(mid + 1, high + 1):  # iterates from mid+1 to high
        total += arr[j]  # adds current element to cumulative sum
        if total > right_sum:  # if current sum is greater than best right sum...
            right_sum = total  # updates best right sum
            max_right = j  # update right boundary
    return max_left, max_right, left_sum + right_sum  # returns indices and combined sum

def find_max_subarray(arr, low, high):
    """
    Find the contiguous subarray with the maximum sum using a divide-and-conquer approach.

    Parameters:
      arr  - List of daily expense totals.
      low  - Starting index of the subarray.
      high - Ending index of the subarray.

    Returns:
      A tuple (start_index, end_index, sum) for the subarray with the maximum sum.
    """
    if low == high:  # base case: if only one element...
        return low, high, arr[low]  # that element is the maximum subarray
    else:
        mid = (low + high) // 2  # calculates middle index
        left_low, left_high, left_sum = find_max_subarray(arr, low, mid)  # recursively finds maximum subarray in left half
        right_low, right_high, right_sum = find_max_subarray(arr, mid + 1, high)  # recursively finds maximum subarray in right half
        cross_low, cross_high, cross_sum = find_max_crossing_subarray(arr, low, mid, high)  # finds maximum subarray that crosses midpoint

        # determines which subarray (left, right, or crossing) has highest sum
        if left_sum >= right_sum and left_sum >= cross_sum:
            return left_low, left_high, left_sum
        elif right_sum >= left_sum and right_sum >= cross_sum:
            return right_low, right_high, right_sum
        else:
            return cross_low, cross_high, cross_sum

def max_spending_period():
    """
    Analyze the maximum spending period over the entire expense history.
    Groups expenses by date, calculates daily totals, and uses the divide-and-conquer algorithm
    to find the contiguous period with the highest spending.
    """
    if not os.path.exists(CSV_FILE):  # checks if CSV file exists
        print("No expenses recorded yet.")  # informs user if no data available
        return  # Exit the function.

    daily_expenses = {}  # initializes a dictionary to store total spending per date
    with open(CSV_FILE, 'r') as csvfile:  # opens CSV file in read mode
        reader = csv.reader(csvfile)  # creates CSV reader object
        header = next(reader, None)  # skips header row
        for row in reader:  # processes each expense record
            if len(row) < 4:  # skips incomplete records
                continue
            date, _, _, amount = row  # extracts date and amount
            try:
                amt = float(amount)  # converts amount to a float
            except ValueError:
                continue  # skips records with invalid amounts
            daily_expenses[date] = daily_expenses.get(date, 0) + amt  # sums expenses for each date

    sorted_dates = sorted(daily_expenses.keys())  # sorts dates in chronological order
    if not sorted_dates:  # if no dates available...
        print("No expense data available for analysis.")  # informs the user.
        return  # exit the function
    amounts = [daily_expenses[date] for date in sorted_dates]  # creates list of daily totals

    # Uses divide-and-conquer algorithm to find maximum spending period
    low_idx, high_idx, max_sum = find_max_subarray(amounts, 0, len(amounts) - 1)
    start_date = sorted_dates[low_idx]  # determines start date of period
    end_date = sorted_dates[high_idx]  # determines end date of period

    print("\nMaximum Spending Period Analysis (Entire History):")  # prinst header
    print(f"From {start_date} to {end_date}, total spending was: ${max_sum:.2f}\n")  # displays period and total spending

def custom_spending_period():
    """
    Analyze spending over a custom date range specified by the user.
    Prompts for a start date and end date, filters expense data accordingly, and
    uses the divide-and-conquer algorithm to determine the maximum spending period within that range.
    """
    if not os.path.exists(CSV_FILE):  # checks if CSV file exists
        print("No expenses recorded yet.")  # informs user if no data available
        return  # exits function

    daily_expenses = {}  # initializes a dictionary for daily totals
    with open(CSV_FILE, 'r') as csvfile:  # opens CSV file for reading
        reader = csv.reader(csvfile)  # creates a CSV reader.
        header = next(reader, None)  # skips header row
        for row in reader:  # processes each expense record
            if len(row) < 4:  # skips incomplete rows
                continue
            date, _, _, amount = row  # extracts date and amount
            try:
                amt = float(amount)  # converts amount to a float
            except ValueError:
                continue  # skips records with invalid amounts
            daily_expenses[date] = daily_expenses.get(date, 0) + amt  # sums expenses for each date

    # prompts user for custom date range
    start_date_input = input("Enter the start date (YYYY-MM-DD) [press Enter to use earliest date]: ").strip()
    end_date_input = input("Enter the end date (YYYY-MM-DD) [press Enter to use latest date]: ").strip()

    if not daily_expenses:  # if no expense data available...
        print("No expense data available for analysis.")  # informs user
        return  # exits function

    sorted_dates = sorted(daily_expenses.keys())  # sorts all available dates
    # uses earliest date if no start date provided
    if start_date_input == "":
        start_date = sorted_dates[0]
    else:
        start_date = start_date_input
    # uses latest date if no end date provided
    if end_date_input == "":
        end_date = sorted_dates[-1]
    else:
        end_date = end_date_input

    # filters expense data to include only dates within specified range
    filtered_expenses = {date: total for date, total in daily_expenses.items() if start_date <= date <= end_date}
    if not filtered_expenses:  # if no data falls within range...
        print("No expense data found in the specified date range.")  # informs user
        return  # exits function

    filtered_sorted_dates = sorted(filtered_expenses.keys())  # sorts filtered dates
    amounts = [filtered_expenses[date] for date in filtered_sorted_dates]  # builds list of totals for range

    # uses divide-and-conquer algorithm on filtered data
    low_idx, high_idx, max_sum = find_max_subarray(amounts, 0, len(amounts) - 1)
    period_start = filtered_sorted_dates[low_idx]  # determines start date of period
    period_end = filtered_sorted_dates[high_idx]  # determines end date of period

    print("\nCustom Spending Period Analysis:")  # prints header
    print(f"From {period_start} to {period_end}, total spending was: ${max_sum:.2f}\n")  # displays period and total spending

def analyze_spending_period():
    """
    Provide a submenu for analyzing the spending period.
    The user can choose to analyze the entire history or a custom date range.
    """
    print("\nAnalyze Spending Period:")
    print("1. Entire Expense History")  # option for full history analysis
    print("2. Custom Date Range")  # option for custom date range analysis
    sub_choice = input("Enter your choice (1 or 2): ").strip()  # prompts user for sub-choice

    if sub_choice == "1":  # if user chooses entire history...
        max_spending_period()  # calls function to analyze entire history
    elif sub_choice == "2":  # if user chooses custom date range...
        custom_spending_period()  # calls function for custom date range analysis
    else:
        print("Invalid choice. Returning to main menu.")  # informs user if input is invalid

def suggest_expense_reductions():
    """
    Provide expense reduction suggestions using a greedy algorithm.
    Prompts for a target savings amount and selects the highest expenses until the target is met.
    """
    try:
        target = float(input("Enter your target savings amount: ").strip())  # prompts for savings target
    except ValueError:
        print("Invalid input. Please enter a numeric value.")  # informs if input is invalid
        return  # exits function.

    if not os.path.exists(CSV_FILE):  # checks if expense data exists
        print("No expenses recorded yet.")  # informs user
        return  # exits function

    expenses = []  # initializes list to store expense records
    with open(CSV_FILE, 'r') as csvfile:  # opens CSV file in read mode
        reader = csv.reader(csvfile)  # creates CSV reader
        header = next(reader, None)  # skips header row
        for row in reader:  # processes each expense record
            if len(row) < 4:  # skips incomplete rows
                continue
            date, category, description, amount = row  # unpacks record
            try:
                amt = float(amount)  # converts amount to a float
            except ValueError:
                continue  # skips records with invalid amounts
            expenses.append((date, category, description, amt))  # adds expense as a tuple to list

    if not expenses:  # if no expense records found...
        print("No expenses available for suggestions.")  # informs user
        return  # exits function

    # sorts expenses in descending order by amount (largest first)
    expenses_sorted = sorted(expenses, key=lambda x: x[3], reverse=True)
    suggestion = []  # initializes list for suggested expense cuts
    current_sum = 0  # initializes variable to accumulate sum of selected expenses

    for expense in expenses_sorted:  # iterates over each expense
        if current_sum >= target:  # if the accumulated sum meets or exceeds target...
            break  # stops adding further expenses
        suggestion.append(expense)  # otherwise, adds expense to suggestions
        current_sum += expense[3]  # increases cumulative sum

    print("\nExpense Reduction Suggestions (Greedy Approach):")  # prints header
    print(f"To reduce your spending by at least ${target:.2f}, consider reviewing these expenses:")
    for date, category, description, amt in suggestion:  # iterates over each suggested expense
        print(f"  {date} | {category} | {description} | ${amt:.2f}")  # prints expense details
    print(f"Total reduction if cut: ${current_sum:.2f}\n")  # displays total suggested reduction
    if current_sum < target:  # if accumulated reduction is less than target...
        print("Note: Even if you remove all these expenses, the target is not met. Consider revising your target or reviewing all expenses.")

def main():
    """
    Main function to run the enhanced expense tracker.
    Displays a menu with options to add/view expenses, view summaries, analyze spending periods (with a submenu),
    suggest expense reductions, and exit the application.
    """
    initialize_file()  # ensures CSV file is initialized
    while True:  # begins main loop to display menu continuously
        print("\nEnhanced Expense Tracker Menu:")  # displays main menu header
        print("1. Add Expense")  # option to add new expense
        print("2. View Expenses")  # option to display all expenses
        print("3. View Summary")  # option to display summary of expenses
        print("4. Analyze Spending Period")  # option to analyze spending periods (submenu provided)
        print("5. Suggest Expense Reductions")  # option to get expense reduction suggestions
        print("6. Exit")  # option to exit application
        choice = input("Enter your choice (1-6): ").strip()  # prompts user for their selection

        if choice == "1":  # if user selects option 1...
            add_expense()  # calls function to add expense
        elif choice == "2":  # if user selects option 2...
            view_expenses()  # calls function to view expenses
        elif choice == "3":  # if user selects option 3...
            view_summary()  # calls function to view summary
        elif choice == "4":  # if user selects option 4...
            analyze_spending_period()  # calls function that provides submenu for spending period analysis
        elif choice == "5":  # if user selects option 5...
            suggest_expense_reductions()  # calls function to suggest expense reductions
        elif choice == "6":  # if user selects option 6...
            print("Goodbye!")  # prints farewell message
            break  # exits loop and ends program
        else:  # if user's choice does not match any valid option...
            print("Invalid choice. Please select a valid option.")  # informs user

if __name__ == "__main__":  # checks if this script is executed directly
    main()  # calls main function to start application



Enhanced Expense Tracker Menu:
1. Add Expense
2. View Expenses
3. View Summary
4. Analyze Spending Period
5. Suggest Expense Reductions
6. Exit
